## Stokes Flow Shear Margin Model

#### Step 0: Let's Import the Necessary Packages

In [5]:
from dolfinx import default_scalar_type
from dolfinx.fem import (
    Constant,
    dirichletbc,
    Function,
    functionspace,
    assemble_scalar,
    form,
    locate_dofs_geometrical,
    locate_dofs_topological,
)
from dolfinx.fem.petsc import LinearProblem
from dolfinx.io import XDMFFile
from dolfinx.io import gmshio
from dolfinx.mesh import create_unit_square, locate_entities
from dolfinx.plot import vtk_mesh
# from dolfinx.io import gmsh

from ufl import SpatialCoordinate, TestFunction, TrialFunction, dx, grad, inner

from mpi4py import MPI

import meshio
import gmsh
import numpy as np
import pyvista
import pandas as pd
import sys
import math

print('imports successful')

imports successful


#### Step 1: Upload and Initialize Data, Import or Create Meshing, and Define Domains

In [9]:
df = pd.read_csv(r"Fenics_model_data_2D.csv")

X_prime = np.reshape(df["X'"],shape = (50,100))
Y_prime = np.reshape(df["Y'"],shape = (50,100))
Surface = np.reshape(df['Surface'],shape = (50,100))
Bed = np.reshape(df['Bed'],shape = (50,100))

dy, dx = np.gradient(matrix)

df.head()

array([[750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742],
       [750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742],
       [750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742],
       ...,
       [750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742],
       [750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742],
       [750.61706742, 750.61706742, 750.61706742, ..., 750.61706742,
        750.61706742, 750.61706742]], shape=(50, 100))

In [7]:
import meshio

mesh_name = "test"

msh = meshio.read(mesh_name+".msh")

print(msh)

# tet_data = msh.cell_data_dict["gmsh:geometrical"]["tetra"]
# meshio.write(mesh_name+".xdmf",
#     meshio.Mesh(points=msh.points,
#         cells={"tetra": msh.cells_dict["tetra"]},
#         cell_data={"dom_marker": [tet_data]}
#     )
# )

# tri_data = msh.cell_data_dict["gmsh:geometrical"]["triangle"]
# meshio.write(mesh_name+"_surf.xdmf",
#     meshio.Mesh(points=msh.points,
#         cells={"triangle": msh.cells_dict["triangle"]},
#         cell_data={"bnd_marker": [tri_data]}
#     )
# )




<meshio mesh object>
  Number of points: 10787
  Number of cells:
    vertex: 1
    vertex: 1
    vertex: 1
    vertex: 1
    vertex: 1
    vertex: 1
    vertex: 1
    vertex: 1
    line: 21
    line: 21
    line: 21
    line: 21
    line: 20
    line: 20
    line: 20
    line: 20
    line: 28
    line: 28
    line: 30
    line: 30
    triangle: 1080
    triangle: 944
    triangle: 1361
    triangle: 1423
    triangle: 1449
    triangle: 1413
    tetra: 55012
  Cell sets: gmsh:bounding_entities
  Point data: gmsh:dim_tags
  Cell data: gmsh:geometrical


In [50]:
from dolfinx import io

# Read the XDMF file
with io.XDMFFile("test.xdmf") as infile:
    mesh = infile.read_mesh()

TypeError: __init__(): incompatible function arguments. The following argument types are supported:
    1. __init__(self, comm: MPICommWrapper, filename: str | os.PathLike, file_mode: str, encoding: dolfinx.cpp.io.XDMFFile.Encoding = Encoding.HDF5) -> None

Invoked with types: dolfinx.io.utils.XDMFFile, str

In [52]:
def terrain_mesher(Nx, Ny, Surface, Zmin, Name, Plot = True):

    gmsh.initialize()
    gmsh.model.add("test")

    # Helper function to return a node tag given two indices i and j:
    def tag(i, j):
        return i * (Ny + 1) + j + 1
    
    # The x, y, z coordinates of all the nodes:
    coords = []
    
    # The tags of the corresponding nodes:
    nodes = []
    
    # The connectivities of the triangle elements (3 node tags per triangle) on the
    # terrain surface:
    tris = []
    
    # The connectivities of the line elements on the 4 boundaries (2 node tags
    # for each line element):
    lin = [[], [], [], []]
    
    # The connectivities of the point elements on the 4 corners (1 node tag for each
    # point element):
    pnt = [tag(0, 0), tag(Nx, 0), tag(Nx, Ny), tag(0, Ny)]
    
    for i in range(Nx + 1):
        for j in range(Ny + 1):
            nodes.append(tag(i, j))
            coords.extend([
                float(i) / Nx,
                float(j) / Ny, 
                Surface[max(i - 1, 0), max(j - 1, 0)]])
            
            if (i == 1 and j == 1) or (i == Nx and j == Ny):
                print('Surface Value:',Surface_N[i - 1, j - 1])
                print('i:', i, '\n j:', j)
                print('Coords Z-Value',coords[3 * tag(i, j) - 1])
            if i > 0 and j > 0:
                tris.extend([tag(i - 1, j - 1), tag(i, j - 1), tag(i - 1, j)])
                tris.extend([tag(i, j - 1), tag(i, j), tag(i - 1, j)])   
            if (i == 0 or i == Nx) and j > 0:
                lin[3 if i == 0 else 1].extend([tag(i, j - 1), tag(i, j)])  
            if (j == 0 or j == Ny) and i > 0:
                lin[0 if j == 0 else 2].extend([tag(i - 1, j), tag(i, j)])
    
    # Create 4 discrete points for the 4 corners of the terrain surface:
    for i in range(4):
        gmsh.model.addDiscreteEntity(0, i + 1)
    
    gmsh.model.setCoordinates(1, 0, 0, coords[3 * tag(0, 0) - 1])
    gmsh.model.setCoordinates(2, 1, 0, coords[3 * tag(Nx, 0) - 1])
    gmsh.model.setCoordinates(3, 1, 1, coords[3 * tag(Nx, Ny) - 1])
    gmsh.model.setCoordinates(4, 0, 1, coords[3 * tag(0, Ny) - 1])

    # Create 4 discrete bounding curves, with their boundary points:
    for i in range(4):
        gmsh.model.addDiscreteEntity(1, i + 1, [i + 1, i + 2 if i < 3 else 1])
    
    # Create one discrete surface, with its bounding curves:
    gmsh.model.addDiscreteEntity(2, 1, [1, 2, 3, 4])
    
    # Add all the nodes on the surface (for simplicity... see below):
    gmsh.model.mesh.addNodes(2, 1, nodes, coords)
    
    # Add point elements on the 4 points, line elements on the 4 curves, and
    # triangle elements on the surface:
    for i in range(4):
        
        # Type 15 for point elements:
        gmsh.model.mesh.addElementsByType(i + 1, 15, [], [pnt[i]])
        
        # Type 1 for 2-node line elements:
        gmsh.model.mesh.addElementsByType(i + 1, 1, [], lin[i])
    
    # Type 2 for 3-node triangle elements:
    gmsh.model.mesh.addElementsByType(1, 2, [], tris)
    
    # Reclassify the nodes on the curves and the points (since we put them all on
    # the surface before with `addNodes' for simplicity)
    gmsh.model.mesh.reclassifyNodes()
    
    # Create a geometry for the discrete curves and surfaces, so that we can remesh
    # them later on:
    gmsh.model.mesh.createGeometry()
    
    # Note that for more complicated meshes, e.g. for on input unstructured STL
    # mesh, we could use `classifySurfaces()' to automatically create the discrete
    # entities and the topology; but we would then have to extract the boundaries
    # afterwards.
    
    p1 = gmsh.model.geo.addPoint(0, 0, Zmin)
    p2 = gmsh.model.geo.addPoint(1, 0, Zmin)
    p3 = gmsh.model.geo.addPoint(1, 1, Zmin)
    p4 = gmsh.model.geo.addPoint(0, 1, Zmin)
    c1 = gmsh.model.geo.addLine(p1, p2)
    c2 = gmsh.model.geo.addLine(p2, p3)
    c3 = gmsh.model.geo.addLine(p3, p4)
    c4 = gmsh.model.geo.addLine(p4, p1)
    c10 = gmsh.model.geo.addLine(p1, 1)
    c11 = gmsh.model.geo.addLine(p2, 2)
    c12 = gmsh.model.geo.addLine(p3, 3)
    c13 = gmsh.model.geo.addLine(p4, 4)
    ll1 = gmsh.model.geo.addCurveLoop([c1, c2, c3, c4])
    s1 = gmsh.model.geo.addPlaneSurface([ll1])
    ll3 = gmsh.model.geo.addCurveLoop([c1, c11, -1, -c10])
    s3 = gmsh.model.geo.addPlaneSurface([ll3])
    ll4 = gmsh.model.geo.addCurveLoop([c2, c12, -2, -c11])
    s4 = gmsh.model.geo.addPlaneSurface([ll4])
    ll5 = gmsh.model.geo.addCurveLoop([c3, c13, 3, -c12])
    s5 = gmsh.model.geo.addPlaneSurface([ll5])
    ll6 = gmsh.model.geo.addCurveLoop([c4, c10, 4, -c13])
    s6 = gmsh.model.geo.addPlaneSurface([ll6])
    sl1 = gmsh.model.geo.addSurfaceLoop([s1, s3, s4, s5, s6, 1])
    v1 = gmsh.model.geo.addVolume([sl1])
    gmsh.model.geo.synchronize()
    
    # Set this to True to build a fully hex mesh:
    #transfinite = True
    transfinite = False
    transfiniteAuto = False
    
    if transfinite:
        NN = 30
        for c in gmsh.model.getEntities(1):
            gmsh.model.mesh.setTransfiniteCurve(c[1], NN)
        for s in gmsh.model.getEntities(2):
            gmsh.model.mesh.setTransfiniteSurface(s[1])
            gmsh.model.mesh.setRecombine(s[0], s[1])
            gmsh.model.mesh.setSmoothing(s[0], s[1], 100)
        gmsh.model.mesh.setTransfiniteVolume(v1)
    elif transfiniteAuto:
        gmsh.option.setNumber('Mesh.MeshSizeMin', 0.5)
        gmsh.option.setNumber('Mesh.MeshSizeMax', 0.5)
        # setTransfiniteAutomatic() uses the sizing constraints to set the number
        # of points
        gmsh.model.mesh.setTransfiniteAutomatic()
    else:
        gmsh.option.setNumber('Mesh.MeshSizeMin', 0.05)
        gmsh.option.setNumber('Mesh.MeshSizeMax', 0.05)
    
    print("Entities:", gmsh.model.getEntities())
    print("Volumes:", gmsh.model.getEntities(3))
    print("Surfaces:", gmsh.model.getEntities(2))
    print("Curves:", gmsh.model.getEntities(1))
    print("Points:", gmsh.model.getEntities(0))
    
    gmsh.model.mesh.generate(3)
    # gmsh.write('test.msh')
    
    # Launch the GUI to see the results:
    if Plot == True:
        gmsh.fltk.run()

    return gmsh.model
    
    # gmsh.finalize()

Surface_N = Surface / Surface.max()

gmsh_model = terrain_mesher(Nx = 50, Ny = 100, Surface = Surface_N, Zmin = -0.5, Name = "test_mesh", Plot = True)

Surface Value: 0.875907100474463
i: 1 
 j: 1
Coords Z-Value 0.875907100474463
Surface Value: 0.9684459736250504
i: 50 
 j: 100
Coords Z-Value 0.9684459736250504
Info    : Creating geometry of discrete curves...
Info    : Done creating geometry of discrete curves (Wall 0.000225386s, CPU 0.000427s)
Info    : Creating geometry of discrete surfaces...
Entities: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (3, 1)]
Volumes: [(3, 1)]
Surfaces: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6)]
Curves: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12)]
Points: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]
Info    : Done creating geometry of discrete surfaces (Wall 0.182946s, CPU 0.406579s)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Discrete curve)
In

X_ChangeProperty: BadValue (integer parameter out of range for operation) 0x0


In [55]:
# mesh, cell_tags, facet_tags = gmshio.read_from_msh("test.msh", MPI.COMM_WORLD, gdim=3)
# print(mesh.topology.dim)
# print(mesh.geometry.x.shape)

# print(gmsh_model)

mesh = gmshio.model_to_mesh(gmsh_model, MPI.COMM_WORLD, rank = 0, gdim = 3)

# circular_mesh, cell_marker, facet_marker = dolfinx.io.gmshio.model_to_mesh(
#     gmsh_model, MPI.COMM_WORLD, 0, gdim=3)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [56]:
gmsh_model.getEntities(dim=3)

[(3, 1)]

In [57]:
gmsh_model

gmsh.model

In [59]:
# List all 3D elements and their types for each 3D entity
for dim, tag in gmsh.model.getEntities(dim=3):
    elem_types, elem_tags, elem_node_tags = gmsh_model.mesh.getElements(dim, tag)
    print(f"Entity (dim={dim}, tag={tag}):")
    print("  Element types:", elem_types)
    print("  Number of elements per type:", [len(t) for t in elem_tags])


Entity (dim=3, tag=1):
  Element types: [4]
  Number of elements per type: [55012]
